In [ ]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from matplotlib import rc, rcParams

%matplotlib inline

%load_ext autoreload
%autoreload 2

rc('text', usetex=True)
font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 14}
rc('font', **font)
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
model_vals = []
attention_vals = []
overall_scores_rows = []
columns = None

def get_attention_text(attention):
    return {'without_dsa': 'None', 'with_dsa': 'DSA'}[attention]

def iter_results_files(*dir_names):
    for dir_name in dir_names:
        yield from pathlib.Path(dir_name).rglob('eval_results.csv')

for results_file in iter_results_files('eval_orig_2x', 'eval_orig_ga'):
    model_dir = results_file.parent.parent
    attention_dir = model_dir.parent

    model = model_dir.stem
    attention = attention_dir.stem

    model_vals.append(model)
    attention_vals.append(get_attention_text(attention))

    df = pd.read_csv(str(results_file), index_col=0)
    columns = df.columns
    overall_scores = df.iloc[-1]
    overall_scores_rows.append(overall_scores)

df_orig = pd.DataFrame(overall_scores_rows, columns=columns)
df_orig.insert(0, 'model', model_vals)
df_orig.insert(1, 'attention', attention_vals)
df_orig.reset_index(drop=True, inplace=True)
df_orig

In [ ]:
df_orig.rename(
        columns={
            'num_frames': 'frames',
            'num_matches': 'matches',
            'num_switches': 'switches',
            'num_false_positives': 'FP',
            'num_misses': 'FN',
            'num_objects': 'objs',
            'num_predictions': 'preds',
            'num_fragmentations': 'fragms',
            'mostly_tracked': 'MT',
            'partially_tracked': 'PT',
            'precision': 'prec',
            'recall': 'rec',
            'idf1': 'IDF1',
            'mota': 'MOTA',
            'motp': 'MOTP'
        },
        inplace=True
    )
df_orig

In [ ]:
def build_model_iters_mask(df, *iters):
    mask = None
    
    for iter in iters:
        iter_str = f'{iter * 1000:07d}'
        curr_mask = df['model'] == iter_str
        mask = curr_mask if mask is None else mask | curr_mask
    
    return mask

iters_mask = build_model_iters_mask(df_orig, 15, 20, 25, 30, 35, 40, 45)
df = df_orig[iters_mask]
df

In [ ]:
df.to_clipboard(excel=True)

In [ ]:
def plot_tracker_attention_comparison(
    df, x_col, y_col, x_label, y_label, x_units='\%', y_units='\%'
):
    def _build_axis_label(text, units=None):
        label = rf'$\textbf{{{text}}}$'
        if units is not None:
            label += f' [{units}]'
        return label
    
    fig, ax = plt.subplots(figsize=(16, 8), nrows=1, ncols=1)

    base_colors =  list(mcolors.BASE_COLORS.keys())
    unique_models = df['model'].unique().tolist()

    size = 30 ** 2
    for attention, group_attention_df in df.groupby(['attention']):
        marker = 'o' if 'None' in attention else 'D'
        
        for model, group_model_df in group_attention_df.groupby(['model']):
            xs, ys = group_model_df[x_col], group_model_df[y_col]    
            
            curr_color = base_colors[unique_models.index(model)]
            label = rf'${attention}, {str(int(model))}$'
            text = int(group_model_df['switches'].values[0])

            ax.scatter(
                xs, ys, s=size, c=curr_color, label=label, marker=marker,
                linewidth=2, alpha=0.7, antialiased=True
            )

            ax.text(xs, ys, s=text, fontdict={'weight': 'bold', 'size': 19})
    
    x_label_formatted = _build_axis_label(x_label, x_units)
    y_label_formatted = _build_axis_label(y_label, y_units)
    
    ax.set_title(
        r'$\textbf{Tracker Performance Comparison --- ' + 
        rf'Training Iterations and Deformable Siamese Attention --- {x_label} vs. {y_label}}}$'
    )
    ax.set_xlabel(x_label_formatted)
    ax.set_ylabel(y_label_formatted)
    ax.legend(loc='best', markerscale=0.5, fontsize=14)

    fig.tight_layout()

    return fig

fig_mota_motp = plot_tracker_attention_comparison(
    df, 'MOTA', 'MOTP', 'MOTA', 'MOTP'
)
fig_mota_motp.savefig('tracker_cmp_plot_mota_motp.png', dpi=300)

In [ ]:
fig_rec_prec = plot_tracker_attention_comparison(
    df, 'rec', 'prec', 'Recall', 'Precision'
)
fig_rec_prec.savefig('tracker_cmp_plot_rec_prec.png', dpi=300)

In [ ]:
fig_rec_prec = plot_tracker_attention_comparison(
    df, 'rec', 'prec', 'Recall', 'Precision'
)
fig_rec_prec.savefig('tracker_cmp_plot_rec_prec.png', dpi=300)